In [1]:
import networkx as nx
from networkx.algorithms import bipartite

# Loading data pathes and I/O functions from script
from scripts.io import load_movie_titles, load_raw_bipartite, save_projection, load_projection, save_edgelist, projection_path

# Loading reccomendation function
from scripts.recommend import k_recommend_from_list

## Loading titles and projections

In [2]:
title_dict, node_dict = load_movie_titles("movie-titles.txt")
simple_weights_movies = load_projection("simple_weights_movies.p")
rating_allocation_movies = load_projection("rating_allocation_movies.p")

Movie titles loaded.
Projection loaded.
Projection loaded.


# Evaluation
    Cross-validation of movie recommendations from graph with empirical ratings by users.

    For each user:
        1. Sample k liked (above average rating) movies, M.
        2. Get movie recommendations, R ((title, weight) list), for all movie titles, based on M.
        3. Get list of liked movies, L ((title, weight) list) based on rating.
        4. Sort R and L by title, then discard the title from both.
        5. Compute and store spearman rank correlation between R and L.
        
    Output average rank correlation for all users.

In [ ]:
def evaluate(k, liked_movie_title, graph):
    """
    Cross-validation of movie recommendations from graph with empirical ratings of users.

    For each user:
        1. Sample k liked (above average rating) movies, M.
        2. Get movie recommendations, R ((title, weight) list), for all movie titles, based on M.
        3. Get list of liked movies, L ((title, weight) list) based on rating.
        4. Sort R and L by title.
        5. Compute and store spearman rank correlation between R and L.
        
    Outputs average correlation between recommendations and preferences.
    """
    total_correlation = 0

    # Step 1: Sample k liked movies, M
    for user_node in user_nodes:
        liked_movies = set(G.nodes[user_node]['movies'])
        above_average_movies = [movie for movie in liked_movies if G.nodes[movie]['rating'] > G.graph['average_rating']]